# spTEP

- Decay/recharge artifact lasts about 0.4 seconds
- Most central electrode is FC1
- Pulse + decay range is about [-1.5/2, 3(largest)/10(most complete)] ms (literature suggests [-1, 6], [-1/2, 5/10])

In [ ]:
# Imports
import os
from ipywidgets import *
import numpy as np
import mne
from mne.preprocessing import ICA
from mne_icalabel import label_components
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

import utils

# Specify graph rendering method
# %matplotlib widget
plt.switch_backend("TkAgg")

DATASET_PATH = "./dataset"
FILENAME_TEMPLATE = "TMS-EEG-H_02_S1b_{}_{}.vhdr"

spTEP_pre_raw = mne.io.read_raw_brainvision(
    os.path.join(DATASET_PATH, FILENAME_TEMPLATE.format("spTEP", "pre")), preload=True
)
sampling_rate = spTEP_pre_raw.info["sfreq"]
events, event_dict = mne.events_from_annotations(spTEP_pre_raw)
tms_indices = [event[0] for event in events if event[2] == 1]

epochs = mne.Epochs(spTEP_pre_raw, events, event_id=event_dict, tmin=-1, tmax=1, baseline=(None, -0.003))
epochs_average = epochs.average()
psd = spTEP_pre_raw.compute_psd()
psd = psd.drop_channels(["VEOG"])

In [112]:
# Plot topomap to indicate where major activity is, indicating zone of application
psd.plot_topomap(show_names=True, bands=[(0.5, 100)]) # Focus on FC1

converting legacy list-of-tuples input to a dict for the `bands` parameter


<Figure size 1220x1290 with 2 Axes>

![topomap](img/EDA/topomap.png)

In [113]:
# Plot average response of all channels
epochs_average.plot(gfp=True, ylim = dict(eeg=[-15, 15]), xlim=(-0.02, 1))

epochs_average.plot(gfp=True, ylim = dict(eeg=[-15, 15]), xlim=(-0.02, 0.5))

/home/tomasgalle/miniconda3/envs/tms/lib/python3.12/site-packages/mne/viz/evoked.py:576: UserWarning: constrained_layout not applied because axes sizes collapsed to zero.  Try making figure larger or axes decorations smaller.
  fig.canvas.draw()  # for axes plots update axes.
/home/tomasgalle/miniconda3/envs/tms/lib/python3.12/tkinter/__init__.py:861: UserWarning: constrained_layout not applied because axes sizes collapsed to zero.  Try making figure larger or axes decorations smaller.
  func(*args)


<Figure size 1220x1290 with 2 Axes>

![](img/EDA/average_total_long.png)
![](img/EDA/average_total_short.png)

In [114]:
# Plot average response of one channel
# channels = ["Cz", "C1", "C3", "C5"] # Used in paper by doctor Carrette
# channels = ["FC1"] # Found by looking at topomap
channels = ["Cz", "C1", "C3", "FC1"] # Used in paper by doctor Carrette

epochs_average.plot(ylim = dict(eeg=[-10, 15]), xlim=(-0.05, 0.5), picks=channels)

<Figure size 1220x1290 with 2 Axes>

![](img/EDA/average_local.png)

In [115]:
# Plot zoomed in
# epochs_average.plot(gfp=True, ylim = dict(eeg=[-15, 15]), picks=channels, xlim=[-0.02, 0.05])

epochs_average.plot(gfp=False, ylim = dict(eeg=[-150, 150]), xlim=(-0.01, 0.02))

<Figure size 1220x1290 with 2 Axes>

![](img/EDA/average_closeup.png)

In [103]:
# Plot discharge artifact
spTEP_pre_raw.plot(events=events, event_id=event_dict, start=102.5, duration=2, n_channels=1, scalings=dict(eeg=50e-5))

<MNEBrowseFigure size 900x1050 with 4 Axes>

Channels marked as bad:
none


![](img/EDA/single_decay_artifact_crop.png)

# rsEEG

In [117]:
rsEEG_pre_raw = mne.io.read_raw_brainvision(
    os.path.join(DATASET_PATH, FILENAME_TEMPLATE.format("rsEEG", "pre")), preload=True
)

# rsEEG_average = rsEEG_pre_raw.average()

Extracting parameters from ./dataset/TMS-EEG-H_02_S1b_rsEEG_pre.vhdr...
Setting channel info structure...
Reading 0 ... 3984899  =      0.000 ...   796.980 secs...


: 

In [104]:
rsEEG_pre_raw.plot(n_channels=1, scalings=dict(eeg=1e-3))

<MNEBrowseFigure size 900x1050 with 4 Axes>

Channels marked as bad:
none
